In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🧠 Function Approximation and Time Series Modeling\n",
    "\n",
    "**Author:** Your Name  \n",
    "**Date:** 2025-08-07\n",
    "\n",
    "This notebook demonstrates:\n",
    "- 1D function approximation using synthetic data\n",
    "- Time series modeling of a biological dynamic system (Lotka–Volterra equations)\n",
    "- Neural network regression using `scikit-learn`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📦 Imports\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy.integrate import solve_ivp\n",
    "from sklearn.neural_network import MLPRegressor\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.metrics import mean_squared_error"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔹 1D Synthetic Data\n",
    "\n",
    "We’ll create a 1D training dataset to use for function approximation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "x1_data = [-12, -10, -5, -2.5, 2, 4, 6, 7.5]\n",
    "Xtrain_1D = np.array(x1_data).reshape(-1, 1)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎯 Target Functions\n",
    "\n",
    "We define two functions:\n",
    "- A sinusoidal + quadratic function\n",
    "- A sum of three Gaussians"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sinusoidal + Quadratic function\n",
    "def Sin_Square_1D(X, noise=False):\n",
    "    y = np.sin(X) + (X/5)**2\n",
    "    if noise:\n",
    "        n = np.random.normal(0, 0.03, X.shape[0])\n",
    "        return (y[:, 0] * (1 + n))\n",
    "    return y[:, 0]\n",
    "\n",
    "# Sum of 3 Gaussians\n",
    "def gaus3_func(X, noise=False):\n",
    "    gaus_params = [[1.0, 2.0, -8.0], [1.5, 1.5, 0.0], [1.4, 1.0, 5.0]]\n",
    "    result = np.zeros(X.shape[0])\n",
    "    for a, std, mean in gaus_params:\n",
    "        result += a / (std * 2.5) * np.exp(-0.5 * ((X[:, 0] - mean)/std)**2)\n",
    "    if noise:\n",
    "        result += np.random.normal(0, 0.01, X.shape[0])\n",
    "    return -result"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Visualize the Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "Ytrain_SS1D = Sin_Square_1D(Xtrain_1D)\n",
    "Ytrain_3G = gaus3_func(Xtrain_1D)\n",
    "\n",
    "# Plot\n",
    "plt.figure(figsize=(10, 4))\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.title(\"Sin + Square Function\")\n",
    "plt.plot(Xtrain_1D, Ytrain_SS1D, 'bo-')\n",
    "plt.grid(True)\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.title(\"Sum of 3 Gaussians\")\n",
    "plt.plot(Xtrain_1D, Ytrain_3G, 'ro-')\n",
    "plt.grid(True)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🤖 MLPRegressor: Function Approximation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Scale inputs\n",
    "scaler = StandardScaler()\n",
    "X_scaled = scaler.fit_transform(Xtrain_1D)\n",
    "\n",
    "# Train MLP on Sin_Square_1D\n",
    "mlp = MLPRegressor(hidden_layer_sizes=(20, 20), max_iter=1000, random_state=1)\n",
    "mlp.fit(X_scaled, Ytrain_SS1D)\n",
    "\n",
    "# Predict on a dense grid\n",
    "X_dense = np.linspace(-15, 10, 200).reshape(-1, 1)\n",
    "X_dense_scaled = scaler.transform(X_dense)\n",
    "Y_pred = mlp.predict(X_dense_scaled)\n",
    "Y_true = Sin_Square_1D(X_dense)\n",
    "\n",
    "# Plot\n",
    "plt.figure(figsize=(8, 5))\n",
    "plt.plot(X_dense, Y_true, label=\"True Function\", color='black')\n",
    "plt.plot(X_dense, Y_pred, label=\"MLP Prediction\", linestyle='--')\n",
    "plt.scatter(Xtrain_1D, Ytrain_SS1D, color='red', label=\"Training Points\")\n",
    "plt.title(\"MLP Approximation of Sin + Square\")\n",
    "plt.xlabel(\"x\")\n",
    "plt.ylabel(\"y\")\n",
    "plt.grid(True)\n",
    "plt.legend()\n",
    "plt.show()\n",
    "\n",
    "# Evaluate\n",
    "print(\"MSE:\", mean_squared_error(Y_true, Y_pred))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔄 Lotka–Volterra Predator-Prey Model\n",
    "This models the population dynamics of a prey and predator species over time."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define the equations\n",
    "def lotka_volterra(t, z, alpha=1.5, beta=1.0, delta=0.75, gamma=1.0):\n",
    "    x, y = z\n",
    "    dxdt = alpha * x - beta * x * y\n",
    "    dydt = delta * x * y - gamma * y\n",
    "    return [dxdt, dydt]\n",
    "\n",
    "# Initial conditions and time span\n",
    "z0 = [10, 5]  # initial prey and predator\n",
    "t_span = (0, 15)\n",
    "t_eval = np.linspace(*t_span, 300)\n",
    "sol = solve_ivp(lotka_volterra, t_span, z0, t_eval=t_eval)\n",
    "t = sol.t\n",
    "x, y = sol.y"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot the time series\n",
    "plt.figure(figsize=(8, 5))\n",
    "plt.plot(t, x, label='Prey')\n",
    "plt.plot(t, y, label='Predator')\n",
    "plt.title(\"Predator-Prey Dynamics\")\n",
    "plt.xlabel(\"Time\")\n",
    "plt.ylabel(\"Population\")\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧩 Extensions and Ideas\n",
    "\n",
    "- Train a neural network to predict future population given history\n",
    "- Add noise to the data and test robustness\n",
    "- Try Gaussian Process Regression for function learning\n",
    "- Use real ecological data for comparison"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": ""
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined